<a href="https://colab.research.google.com/github/aparokshith/MovieBuddy/blob/main/MovieBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions
spark = SparkSession.builder.getOrCreate()

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')
#%cd /gdrive

In [4]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
! kaggle datasets download grouplens/movielens-20m-dataset

movielens-20m-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:

! unzip /content/movielens-20m-dataset.zip

Archive:  /content/movielens-20m-dataset.zip
replace genome_scores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [7]:
ratings_df = spark.read.csv('/content/rating.csv',header = True)
ratings_df.show( n =5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



### Data Cleaning


1.   Remove rows with no Rating
2.   Filter out movies or users with few ratings



In [8]:
ratings_df = ratings_df.filter(functions.col('rating').isNotNull())

In [9]:
# Filter out movies or users with few ratings
min_ratings = 50
movies_df = ratings_df.groupBy('movieId').count().filter(functions.col('count') >= min_ratings)
movies_df = movies_df.select('movieId')
ratings_df = ratings_df.join(movies_df, on='movieId', how='inner')
users_df = ratings_df.groupBy('userId').count().filter(functions.col('count') >= min_ratings)
users_df = users_df.select('userId')
ratings_df = ratings_df.join(users_df, on='userId', how='inner')

In [10]:
ratings_df.printSchema()
ratings_df = ratings_df.withColumn("userId", ratings_df["userId"].cast('int'))
ratings_df = ratings_df.withColumn("movieId", ratings_df["movieId"].cast('int'))
ratings_df = ratings_df.withColumn("rating", ratings_df["rating"].cast('float'))
ratings_df.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)



In [11]:
# Split the data into training and test sets
(train_df, test_df) = ratings_df.randomSplit([0.8, 0.2])
train_df = train_df.limit(100000)

In [12]:
# Define the ALS model
als = ALS(rank=50, maxIter=10, regParam=0.1, userCol='userId', itemCol='movieId', ratingCol='rating',
          coldStartStrategy='drop')

In [13]:
# Fit the ALS model to the training data
model = als.fit(train_df)

In [14]:
predictions_df = model.transform(test_df)

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions_df)
print(f'Root mean squared error: {rmse:.4f}')

Root mean squared error: 0.9066


In [16]:
# Step 6: Recommend similar users based on their ratings on movies
# Select a target user
target_user = 1

# Predict the ratings that the target user would give to all movies
target_user_predictions_df = model.recommendForUserSubset(spark.createDataFrame([(target_user, )], ['userId']), ratings_df.count())



In [17]:
# Find the users who have similar ratings patterns to the target user
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

def to_vector(row):
    return Vectors.dense(row)
    
to_vector_udf = udf(to_vector, VectorUDT())

In [18]:
target_user_predictions_df.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [19]:
target_user_predictions_df.show(3, truncate=False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
target_user_predictions = target_user_predictions_df.select(to_vector_udf(functions.collect_list('recommendations.rating')).alias('target_user_ratings'))

other_users_predictions_df = model.recommendForAllUsers(ratings_df.count()).select(to_vector_udf(functions.collect_list('recommendations.rating')).alias('other_users_ratings'))


Steps left:


1.   Calculate the cosine similarity between the target user predictions and other's prediction
2.   Sort and display top 10 similar users



In [ ]:
#similarity_df = target_user_predictions_df.crossJoin(other_users_predictions_df).withColumn('similarity', functions.cosine_similarity(functions.col('target_user_ratings'), functions.col('other_users_ratings')))

#similar_users_df = similarity_df.sort(functions.desc('similarity'))

In [ ]:
#similar_users_df.show(10)